Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [8]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range


def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])


print("GPUs:", len(tf.config.list_physical_devices('GPU')))

GPUs: 1


First reload the data we generated in `1_notmnist.ipynb`.

In [18]:
pickle_file = 'data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [19]:
image_size = 28
num_labels = 10

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


Train a multinomial logistic regression using simple gradient descent.

In [30]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
num_steps = 801
lr = 0.9

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
      tf.random.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(lr).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
      tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    

with tf.compat.v1.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.compat.v1.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.983480
Training accuracy: 6.2%
Validation accuracy: 10.3%
Loss at step 100: 1.933605
Training accuracy: 74.4%
Validation accuracy: 72.4%
Loss at step 200: 1.533396
Training accuracy: 77.0%
Validation accuracy: 73.9%
Loss at step 300: 1.314265
Training accuracy: 78.1%
Validation accuracy: 74.5%
Loss at step 400: 1.165076
Training accuracy: 78.8%
Validation accuracy: 75.0%
Loss at step 500: 1.054790
Training accuracy: 79.6%
Validation accuracy: 75.2%
Loss at step 600: 0.969236
Training accuracy: 80.2%
Validation accuracy: 75.6%
Loss at step 700: 0.900951
Training accuracy: 80.9%
Validation accuracy: 75.8%
Loss at step 800: 0.845255
Training accuracy: 81.6%
Validation accuracy: 76.0%
Test accuracy: 83.3%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [37]:
batch_size = 128
num_steps = 3001

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.compat.v1.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.compat.v1.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.compat.v1.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.compat.v1.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
    # Optimizer.
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(lr).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.compat.v1.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.compat.v1.matmul(tf_test_dataset, weights) + biases)
    

with tf.compat.v1.Session(graph=graph) as session:
    tf.compat.v1.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.454388
Minibatch accuracy: 10.2%
Validation accuracy: 16.8%
Test accuracy: 17.2%
Test accuracy: 24.2%
Test accuracy: 32.5%
Test accuracy: 42.4%
Test accuracy: 45.2%
Test accuracy: 50.7%
Test accuracy: 53.9%
Test accuracy: 56.5%
Test accuracy: 60.1%
Test accuracy: 60.8%
Test accuracy: 61.8%
Test accuracy: 63.3%
Test accuracy: 64.0%
Test accuracy: 65.6%
Test accuracy: 65.2%
Test accuracy: 66.8%
Test accuracy: 67.8%
Test accuracy: 67.4%
Test accuracy: 66.2%
Test accuracy: 66.3%
Test accuracy: 67.0%
Test accuracy: 69.5%
Test accuracy: 69.0%
Test accuracy: 69.0%
Test accuracy: 69.9%
Test accuracy: 70.7%
Test accuracy: 71.5%
Test accuracy: 71.9%
Test accuracy: 70.2%
Test accuracy: 72.4%
Test accuracy: 72.9%
Test accuracy: 73.6%
Test accuracy: 71.8%
Test accuracy: 73.9%
Test accuracy: 73.5%
Test accuracy: 73.3%
Test accuracy: 74.4%
Test accuracy: 73.8%
Test accuracy: 74.4%
Test accuracy: 74.8%
Test accuracy: 75.0%
Test accuracy: 75.6%
Test accuracy: 75

Test accuracy: 83.3%
Test accuracy: 82.5%
Test accuracy: 82.9%
Test accuracy: 82.8%
Test accuracy: 81.7%
Test accuracy: 82.1%
Test accuracy: 82.7%
Test accuracy: 83.0%
Test accuracy: 82.6%
Test accuracy: 83.2%
Test accuracy: 82.3%
Test accuracy: 81.8%
Test accuracy: 83.0%
Test accuracy: 83.0%
Test accuracy: 82.8%
Test accuracy: 81.0%
Test accuracy: 82.9%
Test accuracy: 82.8%
Test accuracy: 82.8%
Test accuracy: 83.5%
Test accuracy: 83.8%
Test accuracy: 83.2%
Test accuracy: 83.7%
Test accuracy: 83.5%
Test accuracy: 83.2%
Test accuracy: 83.0%
Test accuracy: 83.1%
Test accuracy: 83.6%
Test accuracy: 83.4%
Test accuracy: 83.7%
Test accuracy: 83.5%
Test accuracy: 82.4%
Test accuracy: 82.8%
Test accuracy: 81.0%
Test accuracy: 81.2%
Test accuracy: 82.7%
Test accuracy: 83.1%
Test accuracy: 83.5%
Test accuracy: 82.7%
Test accuracy: 81.5%
Test accuracy: 83.3%
Test accuracy: 82.5%
Test accuracy: 83.2%
Test accuracy: 83.7%
Test accuracy: 83.5%
Test accuracy: 82.7%
Test accuracy: 83.4%
Test accuracy

Test accuracy: 82.6%
Test accuracy: 84.3%
Test accuracy: 84.6%
Test accuracy: 83.7%
Test accuracy: 84.2%
Test accuracy: 84.7%
Test accuracy: 83.9%
Test accuracy: 80.6%
Test accuracy: 83.4%
Test accuracy: 84.2%
Test accuracy: 84.2%
Test accuracy: 84.1%
Test accuracy: 83.2%
Test accuracy: 84.0%
Test accuracy: 82.7%
Test accuracy: 84.5%
Test accuracy: 84.7%
Test accuracy: 84.1%
Test accuracy: 83.8%
Test accuracy: 84.4%
Test accuracy: 84.3%
Test accuracy: 85.0%
Test accuracy: 84.6%
Test accuracy: 84.2%
Test accuracy: 84.3%
Test accuracy: 84.4%
Test accuracy: 83.7%
Test accuracy: 84.4%
Test accuracy: 84.5%
Test accuracy: 85.0%
Test accuracy: 84.5%
Test accuracy: 83.9%
Test accuracy: 83.9%
Test accuracy: 83.8%
Test accuracy: 83.3%
Test accuracy: 84.4%
Test accuracy: 84.7%
Test accuracy: 83.9%
Test accuracy: 83.9%
Test accuracy: 83.3%
Test accuracy: 84.1%
Test accuracy: 84.9%
Test accuracy: 84.7%
Test accuracy: 82.0%
Test accuracy: 83.8%
Test accuracy: 83.7%
Test accuracy: 84.4%
Test accuracy

Test accuracy: 84.2%
Test accuracy: 85.4%
Test accuracy: 85.5%
Test accuracy: 84.9%
Test accuracy: 85.7%
Test accuracy: 83.4%
Test accuracy: 86.1%
Test accuracy: 85.4%
Test accuracy: 85.0%
Test accuracy: 84.9%
Test accuracy: 83.9%
Test accuracy: 84.5%
Test accuracy: 84.7%
Test accuracy: 84.5%
Test accuracy: 84.5%
Test accuracy: 85.7%
Test accuracy: 85.5%
Test accuracy: 85.2%
Test accuracy: 82.7%
Test accuracy: 84.8%
Test accuracy: 84.3%
Test accuracy: 83.7%
Test accuracy: 85.1%
Test accuracy: 84.7%
Test accuracy: 84.8%
Test accuracy: 84.3%
Test accuracy: 85.2%
Test accuracy: 85.6%
Test accuracy: 84.5%
Test accuracy: 84.0%
Test accuracy: 82.1%
Test accuracy: 84.9%
Test accuracy: 84.8%
Test accuracy: 85.2%
Test accuracy: 84.6%
Test accuracy: 83.4%
Test accuracy: 83.5%
Test accuracy: 85.1%
Test accuracy: 85.4%
Test accuracy: 85.0%
Test accuracy: 85.6%
Test accuracy: 84.8%
Test accuracy: 81.5%
Test accuracy: 82.9%
Test accuracy: 84.1%
Test accuracy: 84.7%
Test accuracy: 85.0%
Test accuracy

Test accuracy: 85.8%
Test accuracy: 85.0%
Test accuracy: 85.8%
Test accuracy: 85.4%
Test accuracy: 85.4%
Test accuracy: 84.4%
Test accuracy: 85.4%
Test accuracy: 85.0%
Test accuracy: 85.9%
Test accuracy: 86.2%
Test accuracy: 84.9%
Test accuracy: 85.4%
Test accuracy: 85.9%
Test accuracy: 85.6%
Test accuracy: 83.1%
Test accuracy: 85.7%
Test accuracy: 82.7%
Test accuracy: 85.5%
Test accuracy: 85.3%
Test accuracy: 84.4%
Test accuracy: 79.3%
Test accuracy: 79.9%
Test accuracy: 83.1%
Test accuracy: 85.8%
Test accuracy: 84.4%
Test accuracy: 83.8%
Test accuracy: 85.5%
Test accuracy: 85.5%
Test accuracy: 82.2%
Test accuracy: 83.2%
Test accuracy: 85.3%
Test accuracy: 84.6%
Test accuracy: 85.8%
Test accuracy: 85.9%
Test accuracy: 86.2%
Test accuracy: 84.9%
Test accuracy: 82.7%
Test accuracy: 83.6%
Test accuracy: 85.7%
Test accuracy: 85.7%
Test accuracy: 86.0%
Test accuracy: 85.6%
Test accuracy: 86.0%
Test accuracy: 85.8%
Test accuracy: 85.8%
Test accuracy: 85.3%
Test accuracy: 85.2%
Test accuracy

Test accuracy: 86.2%
Test accuracy: 85.6%
Test accuracy: 83.7%
Test accuracy: 85.1%
Test accuracy: 86.1%
Test accuracy: 85.8%
Test accuracy: 85.6%
Test accuracy: 85.1%
Test accuracy: 82.4%
Test accuracy: 84.9%
Test accuracy: 85.1%
Test accuracy: 86.4%
Test accuracy: 85.9%
Test accuracy: 86.2%
Test accuracy: 86.6%
Test accuracy: 86.8%
Test accuracy: 84.6%
Test accuracy: 84.8%
Test accuracy: 86.5%
Test accuracy: 85.3%
Test accuracy: 84.5%
Test accuracy: 84.2%
Test accuracy: 84.6%
Test accuracy: 85.4%
Test accuracy: 84.7%
Test accuracy: 85.5%
Test accuracy: 86.0%
Test accuracy: 86.2%
Test accuracy: 85.2%
Test accuracy: 85.6%
Test accuracy: 86.2%
Test accuracy: 85.7%
Test accuracy: 85.7%
Test accuracy: 86.5%
Test accuracy: 85.6%
Test accuracy: 86.7%
Test accuracy: 86.9%
Test accuracy: 86.8%
Test accuracy: 87.1%
Test accuracy: 87.0%
Test accuracy: 85.1%
Test accuracy: 86.4%
Test accuracy: 85.1%
Test accuracy: 86.5%
Test accuracy: 82.8%
Test accuracy: 84.7%
Test accuracy: 86.3%
Test accuracy

Test accuracy: 86.3%
Test accuracy: 83.5%
Test accuracy: 86.0%
Test accuracy: 86.3%
Test accuracy: 86.6%
Test accuracy: 83.9%
Test accuracy: 86.0%
Test accuracy: 86.6%
Test accuracy: 86.6%
Test accuracy: 86.9%
Test accuracy: 86.3%
Test accuracy: 86.6%
Test accuracy: 87.1%
Test accuracy: 86.3%
Test accuracy: 86.7%
Test accuracy: 85.6%
Test accuracy: 86.8%
Test accuracy: 87.1%
Test accuracy: 86.7%
Test accuracy: 86.7%
Test accuracy: 86.6%
Test accuracy: 85.8%
Test accuracy: 85.8%
Test accuracy: 86.7%
Test accuracy: 86.3%
Test accuracy: 86.0%
Test accuracy: 87.0%
Test accuracy: 85.8%
Test accuracy: 85.8%
Test accuracy: 80.2%
Test accuracy: 82.5%
Test accuracy: 86.8%
Test accuracy: 85.6%
Test accuracy: 87.1%
Test accuracy: 86.9%
Test accuracy: 86.6%
Test accuracy: 83.6%
Test accuracy: 86.0%
Test accuracy: 86.4%
Test accuracy: 83.7%
Test accuracy: 87.3%
Test accuracy: 82.8%
Test accuracy: 86.7%
Test accuracy: 83.3%
Test accuracy: 83.6%
Test accuracy: 85.8%
Test accuracy: 86.4%
Test accuracy